# Création d'un algorithme de machine learning sur notre base de données

#### Sommaire :
- Observation du dataset
- Nettoyage du dataset
    - identification des variables pertinentes

In [2]:
#installation et importation des librairies nécessaires
!pip install panda
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scipy

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import csv

#importation de notre base de données
import requests
response=requests.get('https://api.hearthstonejson.com/v1/190920/frFR/cards.collectible.json')
data = response.json()
def save_csv(dictio, attributs, name):
    with open(name, mode='w', newline='', encoding='utf-8') as fichier_csv:
        writer = csv.DictWriter(fichier_csv, fieldnames=attributs)
        writer.writeheader()
        for objet in dictio:
            ligne = {attr: objet.get(attr, '') for attr in attributs}
            writer.writerow(ligne)


save_csv(data,["id","name","artist","cardClass","cost","attack","health","set","type","rarity"], "dataMK1.csv")
df = pd.read_csv("dataMK1.csv")

## Observation du dataset

Regardons déjà à quoi ressemble notre dataset :

In [3]:
df

,id,name,artist,cardClass,cost,attack,health,set,type,rarity
0,AT_001,Lance de flammes,Nutthapon Petchthai,MAGE,5.0,NaN,NaN,TGT,SPELL,COMMON
1,AT_002,Effigie,Tooth,MAGE,3.0,NaN,NaN,TGT,SPELL,RARE
2,AT_003,Héros défunt,Arthur Bozonnet,MAGE,2.0,3.0,2.0,TGT,MINION,RARE
3,AT_004,Déflagration des Arcanes,Gabor Szikszai,MAGE,1.0,NaN,NaN,TGT,SPELL,EPIC
4,AT_005,Métamorphose : sanglier,Mike Sass,MAGE,3.0,NaN,NaN,TGT,SPELL,RARE
...,...,...,...,...,...,...,...,...,...,...
5840,YOP_031,Chevaucheur de crabe,Matt Dixon,NEUTRAL,2.0,1.0,4.0,DARKMOON_FAIRE,MINION,COMMON
5841,YOP_032,Vendeuse d’armures,Maria Trepalina,NEUTRAL,1.0,1.0,3.0,DARKMOON_FAIRE,MINION,RARE
5842,YOP_033,Retour de flammes,Anton Zemskov,WARLOCK,3.0,NaN,NaN,DARKMOON_FAIRE,SPELL,COMMON
5843,YOP_034,Aile noire en fuite,Andrew Hou,NEUTRAL,9.0,9.0,9.0,DARKMOON_FAIRE,MINION,RARE


In [4]:
print("type df :", type(df))
print("taille df :", df.shape)
print("noms des colonnes de df :", df.columns)

qualitative_vars = ["id", "name", "artist", "cardClass", "set", "spell"]
quantitative_vars = [col for col in df.columns if col not in qualitative_vars + ["rarity"]]
target = ["rarity"]

cardClasses = df["cardClass"].value_counts()
artists = df["artist"].value_counts()
sets = df["set"].value_counts()
types = df["type"].value_counts()
rarities = df["rarity"].value_counts()

print("qualitative_vars :", qualitative_vars)
print("quantitative_vars :", quantitative_vars)
print("target : ", target, "\n")
print("\nValeurs prises pour chaque colonne (seulement les colonnes sur lesquelles il n'y a pas trop de cas):")
print("cardClass : ", cardClasses, "\n")
print("artist : ", artists, "\n")
print("set : ", sets, "\n")
print("type : ", types, "\n")
print("rarity : ", rarities, "\n")

type df : <class 'pandas.core.frame.DataFrame'>
taille df : (5845, 10)
noms des colonnes de df : Index(['id', 'name', 'artist', 'cardClass', 'cost', 'attack', 'health', 'set',
       'type', 'rarity'],
      dtype='object')
qualitative_vars : ['id', 'name', 'artist', 'cardClass', 'set', 'spell']
quantitative_vars : ['cost', 'attack', 'health', 'type']
target :  ['rarity'] 


Valeurs prises pour chaque colonne (seulement les colonnes sur lesquelles il n'y a pas trop de cas):
cardClass :  cardClass
NEUTRAL        1675
PRIEST          430
PALADIN         427
WARRIOR         425
WARLOCK         424
MAGE            424
DRUID           421
SHAMAN          419
ROGUE           414
HUNTER          413
DEMONHUNTER     247
DEATHKNIGHT     126
Name: count, dtype: int64 

artist :  artist
James Ryman                   247
Matt Dixon                    235
Alex Horley Orlandelli        206
Zoltan Boros                  203
Anton Zemskov                 139
                             ... 
A. Gimald

Remarques préalables : 
Je ne sais pas si les artistes sont pertinents, mais il y a surement trop d'artistes avec une seule carte dessinée pour pouvoir les exploiter. (Ou, à la rigueur, on fait une catégorie "artistes avec moins de 50 cartes dessinées par lui/elle", par exemple).

Ici, notre but est de prédire la rareté de la carte -> 5 modalités (common, rare, epic, legendary et free)

## Nettoyage du dataset

Pour commencer, on veut un dataset “clean”, c.à.d tel que : 
- on a trouvé des solutions de remplacement pour les NaN
  Q : Est-ce qu'on peut remplacer attack = NaN par -1 ? (et idem pour health).
- on n’a conservé que les variables utiles
- on a une liste des variables qualitatives et quantitatives
- on sait quel score on veut optimiser (en fonction des données qu’on comme base)

### Se débarrasser des NaN

In [6]:
import math
#remplacer les attacks NaN par -1
dfcorrige = df
for i in range (0,df["attack"].shape[0]):
    if math.isnan(dfcorrige["attack"][i]):
        dfcorrige["attack"][i] = -1

#remplacer les attacks NaN par -1
for i in range (0,df["health"].shape[0]):
    if math.isnan(dfcorrige["health"][i]):
        dfcorrige["health"][i] = -1

In [7]:
dfcorrige["attack"]
dfcorrige["health"]

0      -1.0
1      -1.0
2       2.0
3      -1.0
4      -1.0
       ... 
5840    4.0
5841    3.0
5842   -1.0
5843    9.0
5844    3.0
Name: health, Length: 5845, dtype: float64

### Identification des variables pertinentes

Pour identifier les variables pertinentes, quels types d'analyses faut-il réaliser ?

Proposition 1 : Pour variable quantitative et target qualitative (5 modalités), on peut réaliser pour chaque variable un boxplot des valeurs qu'elle prend pour target="common", "rare", "epic", "legendary" et "free".
Q : Peut-on utiliser les histogrammes, les violinplots et les scatterplots de façon pertinente ?

Proposition 2 : Tracer la heatmap des correlations entre chaques variable 

In [10]:
df[quantitative_vars].corr()

KeyError: "['spell'] not in index"

Pourquoi ça ne fonctionne pas ?
Je vais essayer de faire la même fonction que dans le lab2, mais je ne le comprends pas : 

In [ ]:
# Oh no, there seems to be a variable I have stored as quantitative that is actually qualitative

def check_numeric(x):
    """Check if a value is numeric, else prints it.
    """
    try:
        int(x)
    except ValueError:
        print(x)


for var in quantitative_vars:
    if sum(df[var].apply(lambda x: type(x) in [int, float])) != df[var].shape[0]:
        print(f"{var} is not fully numeric.")
        print("Non numeric value is : ")
        df[var].apply(lambda x: check_numeric(x))


# Let's transform it into a numeric variable ... what else could I do ?
df["capture_rate"] = df["capture_rate"].apply(lambda x: x.split(" ")[0] if type(x) not in [int, float] else x).astype(float)

Knn -> obigé de passer sur du one-hot encoding 
Mais bayesien naif et forets -> pas de pb avec les 
Les attack et health -> faire de l'empirique : tester -1 ou 99, voir si ça fonctionne.